

# Thực hành mạng nơ-ron nhân tạo (Neural Network)


## Bài toán:


> **Đánh giá điểm tín dụng sử dụng Neural Network**

Mục tiêu:

- Xây dựng mô hình Neural Network sử dụng thư viện sklearn.
- Áp dụng mô hình để giải bài toán thực tế (đánh giá điểm tín dụng).
- Sử dụng độ đo thích hợp để đánh giá chất lượng mô hình.

Vấn đề:

- Trong lĩnh vực tài chính, bài toán đánh giá tín dụng của người dùng là bài toán phổ biến và quan trọng. Ở đó, ngân hàng sẽ xác định liệu có cho một người dùng nào đó thực hiện khoản vay hay không, dựa trên thông tin về điểm tín dụng. Điểm tín dụng chính là con số phản ánh khả năng hoàn trả của người vay. Nhìn chung, đây là một bài toán phân lọai 2 lớp (good/bad score).
- Ngoài ra, một vấn đề thường gặp phải trong bài toán này đó: số lượng bad credits thường ít hơn nhiều so với số lượng good credits ==> Dữ liệu nhãn lớp mất cân bằng.

Dữ liệu:

- Cho dưới dạng bảng, mỗi hàng là một cá nhân vay, mỗi cột thể hiện một thuộc tính của cá nhân đó.
- Các trường thuộc tính tồn tại ở nhiều dạng: categorical, numeric (discrete / continuous). Thậm chí tên trường có thể được ẩn đi, hay giá trị thật sự được mã hõa để đảm bảo tính bảo mật của khách hàng.
- Tập dữ liệu sẽ sử dụng:  German Credit




Mô hình hóa bài toán: bài toán phân loại nhị phân

- Đâu vào: $N$ vector $D$ chiều (ma trận cỡ $N \times D$), tương ứng $N$ cá thể với $D$ thuộc tính.
- Đầu ra: nhãn $0-1$ (0: bad credit, 1: good credit).

# Các bước làm

In [ ]:
#Nếu chạy trên Google Colab thì cần kết nối với máy chủ trước
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, ShuffleSplit, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier

random_state = 0


## Tìm hiểu dữ liệu

In [ ]:
'''
- Tìm hiểu cấu trúc data tại trang web UCI
- Down các files, đặt trong folder data/
- Làm với phiên bản 'numeric'
'''
data_path = '/content/german.data-numeric'
credit=np.genfromtxt(data_path)
print(credit)
X,y = credit[:,:-1], credit[:,-1]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)


[[ 1.  6.  4. ...  0.  1.  1.]
 [ 2. 48.  2. ...  0.  1.  2.]
 [ 4. 12.  4. ...  1.  0.  1.]
 ...
 [ 4. 12.  2. ...  0.  1.  1.]
 [ 1. 45.  2. ...  0.  1.  2.]
 [ 2. 45.  4. ...  0.  1.  1.]]
(1000, 24) (1000,)
(800, 24) (200, 24)


# Neural network



Ví dụ

Khởi tạo mô hình với 2 tầng ẩn có 30 nút (30, 30) là một tuple biểu diễn số lượng nút ở mỗi tầng ẩn.

from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(30, 30), max_iter=1000)

Gọi phương thức fit để huấn luyện mô hình với dữ liệu huấn luyện X_train là ma trận dữ liệu huấn luyện, y_train là vector nhãn.

model.fit(X_train, y_train)

Bài tập:
- Tạo một mạng neural network một tầng ẩn.
- Viết chương trình so sánh hiệu suất của mô hình neural network trên các tập cross-validation từ đó chọn ra kích thước hidden layer tối ưu. Với kích thước này hãy training lại mô hình với toàn bộ dữ liệu, sau đó đánh giá với f1 score.

In [ ]:

cv_accuracies_by_hidden_layer_size = []
hidden_layer_sizes_values = [(10,), (20,), (30,), (40,), (50,), (60,), (70,), (80,), (90,)] # Các kích thước của tầng ẩn
# Code ở đây:

# Plot
plt.plot(hidden_layer_sizes_values, cv_accuracies_by_hidden_layer_size, label='cv')
plt.legend()
plt.xlabel('hidden layer size')
plt.ylabel('scores')
plt.title('MLP validation curve for hidden_layer_size')
plt.show()


### Hyperparameter tuning

Định nghĩa một parameter grid, tức là một lưới các tham số, để sử dụng trong việc tìm kiếm siêu tham số (hyperparameter tuning) cho mô hình MLP (Multi-Layer Perceptron) trong scikit-learn. Dưới đây là ý nghĩa của các tham số:

- 'hidden_layer_sizes': Là một list chứa các tuple biểu diễn cấu trúc của các tầng ẩn trong mô hình MLP. Ví dụ, (10,) có nghĩa là một tầng ẩn với 10 nút, (20,) là một tầng ẩn khác với 20 nút, và (30,) là một tầng ẩn khác nữa với 30 nút.

- 'activation': Là một list chứa các hàm kích hoạt được sử dụng trong các tầng ẩn. Trong trường hợp này, 'relu' là Rectified Linear Unit và 'logistic' là hàm logistic sigmoid.

- 'alpha': Là một list chứa các giá trị của tham số alpha, là tham số điều chuẩn (regularization) cho mô hình. Các giá trị này đại diện cho mức độ điều chuẩn được áp dụng, giúp tránh overfitting.

- 'max_iter': Là một list chứa số lần lặp tối đa trong quá trình huấn luyện. Trong trường hợp này, chỉ có một giá trị là 1000.

- 'solver': Là một list chứa tên của thuật toán tối ưu hóa sử dụng trong quá
trình huấn luyện. Trong trường hợp này, 'adam' là một thuật toán tối ưu phổ biến cho huấn luyện mạng neural network.

Các tham số trong parameter grid này sẽ được sử dụng trong quá trình tìm kiếm siêu tham số để tìm ra bộ tham số tối ưu cho mô hình MLP.

In [ ]:
# Define the parameter grid for MLP
parameter = {
    'hidden_layer_sizes': [(10,), (20,), (30,)],
    'activation': ['relu', 'logistic'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000],
    'solver': ['adam'],
}

# Sử dụng GridSearchCV với bộ tham số trên để tìm ra estimator tốt nhất (In ra). Huấn luyện và đánh giá mô hình này.